# Information visualization - system B

In [1]:
import pandas as pd
import altair as alt
import panel as pn
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
alt.data_transformers.disable_max_rows()
# read the csv file
predictions = pd.read_csv("dataset/weather_prediction_dataset.csv")

# Parsing cities
locations = [column.split("_")[0] for column in predictions if "_" in column]
locations = pd.Series(locations).unique()
locations[2] = "DE_BILT"
print("locations:\n", locations, "\n")

# Divide data by city
predictions = {
    location: pd.concat(
        [
            predictions[["DATE", "MONTH"]],
            predictions.filter(like=location),
        ],
        axis=1,
    )
    for location in locations
}

# Process column names (remove city prefix)
weather_phenomena = []
for location, prediction in predictions.items():
    prediction.columns = [column.replace(location + "_", "") for column in prediction.columns]
    for column in prediction.columns:
        if column not in weather_phenomena:
            weather_phenomena.append(column)
weather_remove = {'cloud_cover', 'pressure', 'global_radiation', 'sunshine', 'wind_speed', 'wind_gust'}  # Use a set for fast lookup
weather_phenomena = [item for item in weather_phenomena if item not in weather_remove]
weather_phenomena.remove("DATE")
weather_phenomena.remove("MONTH")
print("Weather phenomena:", weather_phenomena)

# Convert DataFrame and handle time
predictions = pd.concat(predictions, names=["location"]).reset_index()
predictions = predictions.drop(columns=["level_1", "MONTH"])
predictions["DATE"] = pd.to_datetime(predictions["DATE"], format="%Y%m%d")
predictions.reset_index(drop=True, inplace=True)  # 确保 DATE 不是索引，以避免 KeyError
predictions = predictions.drop(columns=[col for col in weather_remove if col in predictions.columns])
print(predictions.columns)


# Create a multi-select control and ensure the default value is a list
location_selector = pn.widgets.CrossSelector(
    name="Select Locations",
    options=list(locations),  
    value=["BASEL"],  
    width=800, 
    height=150
)

phenomenon_selector = pn.widgets.Select(
    name="Select Phenomenon", 
    options=weather_phenomena, value="humidity"
)

date_slider = pn.widgets.DateRangeSlider(
    name="Select Date Range",
    start=predictions["DATE"].min(),
    end=predictions["DATE"].max(),
    value=(predictions["DATE"].min(), predictions["DATE"].max()),
    width=1850,
)

# Update the left chart (multiple city colors)
def update_charts(phenomenon, locations, date_range):
    # Convert selected date range to pandas datetime
    date_start = pd.to_datetime(date_range[0])  # Convert start date
    date_end = pd.to_datetime(date_range[1])    # Convert end date

    if locations:  # Avoid errors with empty lists
        filtered_predictions = predictions[predictions["location"].isin(locations)]

    # filtered_predictions = predictions[(predictions["location"] == location)]
    filtered_predictions = filtered_predictions[ (filtered_predictions["DATE"] >= date_start) & (filtered_predictions["DATE"] <= date_end) ]
    
    # **Only keep DATE, location and selected weather phenomenon columns**
    columns_to_keep = ["DATE", "location", phenomenon]
    filtered_predictions = filtered_predictions[columns_to_keep]

    brush = alt.selection_interval()

    # point chart
    chart = (
        alt.Chart(filtered_predictions)
        .mark_point(size=10)
        .encode(
            x=alt.X("DATE:T", title="Date"),
            y=alt.Y(phenomenon, title=phenomenon),
            color=alt.Color("location:N", title="City"),
            shape=alt.Shape("location:N", title="City"),
            tooltip=[
                alt.Tooltip("DATE:T", title="Date"),
                alt.Tooltip("location:N", title="City"),
                alt.Tooltip(f"{phenomenon}:Q", title=f"{phenomenon}")
            ]
        )
        .properties(width=1600, height=260, title=f"Filtered Scatter Plot: {phenomenon}")
        .add_params(brush)
    )

    # bar chart
    sub_chart1 = (
        alt.Chart(filtered_predictions)
        .mark_bar()
        .encode(
            x=alt.X('location:N', title="locations", sort="-x", axis=alt.Axis(titleAngle=0, titleX=-80)),
            y=alt.Y(f"sum({phenomenon}):Q" if phenomenon == 'precipitation' else f'mean({phenomenon}):Q', title=phenomenon),
            color=alt.Color("location:N", title="City"),
        )
        .transform_filter(brush)
        .properties(width=800, height=200, title=f"Filtered Bar Chart: {phenomenon}")
    )


    # line chart
    sub_chart2 = (
        alt.Chart(filtered_predictions)
        .mark_line(size=2)
        .encode(
            x=alt.X("DATE:T", title="Date"),
            y=alt.Y(phenomenon, title=phenomenon),
            color=alt.Color("location:N", title="City"),
        )
        .transform_filter(brush)
        .properties(width=800, height=200, title=f"Filtered Line Chart: {phenomenon}")
    )

    # Overlay `mark_point()` on the line chart so that the tooltip is displayed when the mouse hovers
    points = (
        alt.Chart(filtered_predictions)
        .mark_point(size=10) 
        .encode(
            x=alt.X("DATE:T", title="Date"),
            y=alt.Y(phenomenon, title=phenomenon),
            color=alt.Color("location:N", title="City"),
            tooltip=[
                alt.Tooltip("DATE:T", title="Date"),
                alt.Tooltip("location:N", title="City"),
                alt.Tooltip(f"{phenomenon}:Q", title=f"{phenomenon}")
            ]
        )
        .transform_filter(brush)
    )
    
    # Combining a line chart with a transparent tooltip
    sub_chart2 = sub_chart2 + points

    return chart & (sub_chart1 | sub_chart2)

# Bind chart data
charts = pn.pane.Vega(
    pn.bind(
        update_charts,
        phenomenon=phenomenon_selector,
        locations=location_selector,
        date_range=date_slider,
    )
)

controller = pn.Column(
    pn.Row(location_selector, phenomenon_selector),
    date_slider,
)

# Custom title HTML, white font + left margin
controller_title = '<div style="color: white; padding-left: 15px;">Control Panel</div>'
charts_title = '<div style="color: white; padding-left: 15px;">Data Visualization</div>'

# Set the controller style
controller_card = pn.layout.Card(
    controller, 
    title=controller_title, 
    collapsible=False, 
    width=1880, 
    header_background="#007acc" 
)
controller_card.styles.update({
    "border": "2px solid #007acc", 
    "border-radius": "10px", 
    "margin": "3px", 
})

# Set the style of the chart
charts_card = pn.layout.Card(
    charts, 
    title=charts_title,  
    collapsible=False, 
    width=1880, 
    header_background="#007acc"
)
charts_card.styles.update({
    "border": "2px solid #007acc",
    "border-radius": "10px",
    "margin": "3px",
})

# UI layout
dashboard = pn.Column(
    controller_card,
    charts_card,
    sizing_mode="stretch_both",
    align="center",
)

# Dashboard
dashboard.show(threaded=True)

locations:
 ['BASEL' 'BUDAPEST' 'DE_BILT' 'DRESDEN' 'DUSSELDORF' 'HEATHROW' 'KASSEL'
 'LJUBLJANA' 'MAASTRICHT' 'MALMO' 'MONTELIMAR' 'MUENCHEN' 'OSLO'
 'PERPIGNAN' 'ROMA' 'SONNBLICK' 'STOCKHOLM' 'TOURS'] 

Weather phenomena: ['humidity', 'precipitation', 'temp_mean', 'temp_min', 'temp_max']
Index(['location', 'DATE', 'humidity', 'precipitation', 'temp_mean',
       'temp_min', 'temp_max'],
      dtype='object')


<StoppableThread(Thread-5 (get_server), started 9908)>

Launching server at http://localhost:54609


ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'MessageSent', 'msg_type': 'bokeh_event', 'msg_data': {'type': 'event', 'name': 'vega_event', 'values': {'type': 'map', 'entries': [['model', {'id': 'p1030'}], ['data', {'type': 'map', 'entries': [['type', 'param_10'], ['value', {'type': 'map', 'entries': [['DATE', [948019950000, 954208080000]], ['temp_min', [-15, 17.997596153846153]]]}]]}]]}}}]} 
 error: ValueError("'param_10' is not a parameter of Selection")
Traceback (most recent call last):
  File "c:\Users\54062\anaconda3\Lib\site-packages\bokeh\server\protocol_handler.py", line 94, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\54062\anaconda3\Lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\54062\anaconda3\L